**import**

In [ ]:
import os


from collections import Counter
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np 
import nest_asyncio #to avoid runtime issue
nest_asyncio.apply()
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
nltk.download('punkt') #contents a pre-trained Punkt tokenizer
from sklearn.decomposition import NMF
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from sklearn.decomposition import PCA
import re 
import random 
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import twint
from sklearn.feature_extraction.text import TfidfVectorizer 
from unidecode import unidecode
from wordcloud import WordCloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


**I. SCRAPPING**

In [ ]:
__import__('IPython')

#c= twint.Config()
#c.Pandas = True #on intègre pandas à twint pour pouvoir générer le dataframe lié
#c.Search = "covid"
#c.Since ="2021-01-02"
#c.Until ="2021-12-30"
#c.Store_object = True
#c.Lang ='fr'
#c.Limit = 300000
#c.Store_csv = True
#c.Output = "Tweet-File"


#twint.run.Search(c)

**II. ANALYSE DESCRIPTIVE**

In [ ]:
tweets = pd.read_csv(r'\Text Mining_Adrien\tweets.csv')

In [ ]:
tweets.shape

In [ ]:
tweets.head()

In [ ]:
tweets.columns

In [ ]:
df = tweets[tweets['language']=='fr'] #nouveau dataframe avec seulement les tweets en français
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['word_titles'] = df['hashtags'].apply(lambda x: len(x.split())) #take a look the average number of hashtags in tweets
df.describe()

In [ ]:
df['tweet'].head

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna(axis=1) #removing nan
df.reset_index(drop=True, inplace=True)

In [ ]:
df.isna().sum()

In [ ]:
#pd.options.display.max_rows = None
#pd.options.display.max_colwidth = None

corpus = list(df['tweet'])
#print(corpus)

In [ ]:
len(corpus)

In [ ]:
type(corpus)

In [ ]:
i = 1

for doc in corpus:
    print(f"Le document {i} est de longueur : {len(doc)}")
    i+=1

**III. PREPROCESSING**

In [ ]:
#Tokeninzination
def hashtags(corpus):
    i = 1

    for doc in corpus:
        #liste_mots= word_tokenize(doc)
        #liste_mots = re.split('\W+',doc)
        hash= re.findall(r"#(\w+)", doc)
        print(f"Le document {i} contient {len(hash)} mots")
        print(f"La liste de mots du documents {i} : {hash} \n")
        i+=1
    return hash

print(hashtags(corpus))

In [ ]:
if doc.isdigit() in corpus :
    print(doc)
else : 
    print("Tous les documents sont de type string")

In [ ]:
stopWords = ['a', 'abord', 'absolument', 'afin', 'ah', 'ai', 'aie', 'ailleurs', 'ainsi', 'ait', 'allaient', 'allo', 'allons', 
             'allô', 'alors', 'anterieur', 'anterieure', 'anterieures', 'apres', 'après', 'as', 'assez', 'attendu', 'au', 
             'aucun', 'aucune', 'aujourd', "aujourd'hui", 'aupres', 'auquel', 'aura', 'auraient', 'aurait', 'auront', 'aussi', 
             'autre', 'autrefois', 'autrement', 'autres', 'autrui', 'aux', 'auxquelles', 'auxquels', 'avaient', 'avais', 'avait', 
             'avant', 'avec', 'avoir', 'avons', 'ayant', 'bah', 'bas', 'basee', 'bat', 'beau', 'beaucoup', 'bien', 'bigre', 'boum', 
             'bravo', 'brrr', "c'", 'car', 'ce', 'ceci', 'cela', 'celle', 'celle-ci', 'celle-là', 'celles', 'celles-ci', 'celles-là', 
             'celui', 'celui-ci', 'celui-là', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes', 'ces', 
             'cet', 'cette', 'ceux', 'ceux-ci', 'ceux-là', 'chacun', 'chacune', 'chaque', 'cher', 'chers', 'chez', 'chiche', 'chut', 
             'chère', 'chères', 'ci', 'cinq', 'cinquantaine', 'cinquante', 'cinquantième', 'cinquième', 'clac', 'clic', 'combien', 
             'comme', 'comment', 'comparable', 'comparables', 'compris', 'concernant', 'contre', 'couic', 'crac', 'c’', "d'", 'da', 
             'dans', 'de', 'debout', 'dedans', 'dehors', 'deja', 'delà', 'depuis', 'dernier', 'derniere', 'derriere', 'derrière', 
             'des', 'desormais', 'desquelles', 'desquels', 'dessous', 'dessus', 'deux', 'deuxième', 'deuxièmement', 'devant', 'devers', 
             'devra', 'different', 'differentes', 'differents', 'différent', 'différente', 'différentes', 'différents', 'dire', 
             'directe', 'directement', 'dit', 'dite', 'dits', 'divers', 'diverse', 'diverses', 'dix', 'dix-huit', 'dix-neuf', 
             'dix-sept', 'dixième', 'doit', 'doivent', 'donc', 'dont', 'douze', 'douzième', 'dring', 'du', 'duquel', 'durant', 'dès', 
             'désormais', 'd’', 'effet', 'egale', 'egalement', 'egales', 'eh', 'elle', 'elle-même', 'elles', 'elles-mêmes', 'en', 
             'encore', 'enfin', 'entre', 'envers', 'environ', 'es', 'est', 'et', 'etaient', 'etais', 'etait', 'etant', 'etc', 'etre', 
             'eu', 'euh', 'eux', 'eux-mêmes', 'exactement', 'excepté', 'extenso', 'exterieur', 'fais', 'faisaient', 'faisant', 'fait', 
             'façon', 'feront', 'fi', 'flac', 'floc', 'font', 'gens', 'ha', 'hein', 'hem', 'hep', 'hi', 'ho', 'holà', 'hop', 'hormis', 
             'hors', 'hou', 'houp', 'hue', 'hui', 'huit', 'huitième', 'hum', 'hurrah', 'hé', 'hélas', 'i', 'il', 'ils', 'importe', 
             "j'", 'je', 'jusqu', 'jusque', 'juste', 'j’', "l'", 'la', 'laisser', 'laquelle', 'las', 'le', 'lequel', 'les', 
             'lesquelles', 'lesquels', 'leur', 'leurs', 'longtemps', 'lors', 'lorsque', 'lui', 'lui-meme', 'lui-même', 'là', 'lès', 'l’', 
             "m'", 'ma', 'maint', 'maintenant', 'mais', 'malgre', 'malgré', 'maximale', 'me', 'meme', 'memes', 'merci', 'mes', 'mien', 'mienne', 
             'miennes', 'miens', 'mille', 'mince', 'minimale', 'moi', 'moi-meme', 'moi-même', 'moindres', 'moins', 'mon', 
             'moyennant', 'même', 'mêmes', 'm’', "n'", 'na', 'naturel', 'naturelle', 'naturelles', 'ne', 'neanmoins', 'necessaire', 
             'necessairement', 'neuf', 'neuvième', 'ni', 'nombreuses', 'nombreux', 'non', 'nos', 'notamment', 'notre', 'nous', 'nous-mêmes', 
             'nouveau', 'nul', 'néanmoins', 'nôtre', 'nôtres', 'n’', 'o', 'oh', 'ohé', 'ollé', 'olé', 'on', 'ont', 'onze', 'onzième', 'ore', 
             'ou', 'ouf', 'ouias', 'oust', 'ouste', 'outre', 'ouvert', 'ouverte', 'ouverts', 'où', 'paf', 'pan', 'par', 'parce', 'parfois', 
             'parle', 'parlent', 'parler', 'parmi', 'parseme', 'partant', 'particulier', 'particulière', 'particulièrement', 'pas', 'passé', 
             'pendant', 'pense', 'permet', 'personne', 'peu', 'peut', 'peuvent', 'peux', 'pff', 'pfft', 'pfut', 'pif', 'pire', 'plein', 'plouf', 
             'plus', 'plusieurs', 'plutôt', 'possessif', 'possessifs', 'possible', 'possibles', 'pouah', 'pour', 'pourquoi', 'pourrais', 'pourrait', 
             'pouvait', 'prealable', 'precisement', 'premier', 'première', 'premièrement', 'pres', 'probable', 'probante', 'procedant', 'proche', 
             'près', 'psitt', 'pu', 'puis', 'puisque', 'pur', 'pure', "qu'", 'quand', 'quant', 'quant-à-soi', 'quanta', 'quarante', 'quatorze', 
             'quatre', 'quatre-vingt', 'quatrième', 'quatrièmement', 'que', 'quel', 'quelconque', 'quelle', 'quelles', "quelqu'un", 'quelque', 
             'quelques', 'quels', 'qui', 'quiconque', 'quinze', 'quoi', 'quoique', 'qu’', 'rare', 'rarement', 'rares', 'relative', 'relativement', 
             'remarquable', 'rend', 'rendre', 'restant', 'reste', 'restent', 'restrictif', 'retour', 'revoici', 'revoilà', 'rien', "s'", 'sa', 
             'sacrebleu', 'sait', 'sans', 'sapristi', 'sauf', 'se', 'sein', 'seize', 'selon', 'semblable', 'semblaient', 'semble', 'semblent', 
             'sent', 'sept', 'septième', 'sera', 'seraient', 'serait', 'seront', 'ses', 'seul', 'seule', 'seulement', 'si', 'sien', 'sienne', 
             'siennes', 'siens', 'sinon', 'six', 'sixième', 'soi', 'soi-même', 'soit', 'soixante', 'son', 'sont', 'sous', 'souvent', 'specifique', 
             'specifiques', 'speculatif', 'stop', 'strictement', 'subtiles', 'suffisant', 'suffisante', 'suffit', 'suis', 'suit', 'suivant', 
             'suivante', 'suivantes', 'suivants', 'suivre', 'superpose', 'sur', 'surtout', 's’', "t'", 'ta', 'tac', 'tant', 'tardive', 'te', 
             'tel', 'telle', 'tellement', 'telles', 'tels', 'tenant', 'tend', 'tenir', 'tente', 'tes', 'tic', 'tien', 'tienne', 'tiennes', 
             'tiens', 'toc', 'toi', 'toi-même', 'ton', 'touchant', 'toujours', 'tous', 'tout', 'toute', 'toutefois', 'toutes', 'treize', 'trente', 
             'tres', 'trois', 'troisième', 'troisièmement', 'trop', 'très', 'tsoin', 'tsouin', 'tu', 'té', 't’', 'un', 'une', 'unes', 
             'uniformement', 'unique', 'uniques', 'uns', 'va', 'vais', 'vas', 'vers', 'via', 'vif', 'vifs', 'vingt', 'vivat', 'vive', 'vives', 
             'vlan', 'voici', 'voilà', 'vont', 'vos', 'votre', 'vous', 'vous-mêmes', 'vu', 'vé', 'vôtre', 'vôtres', 'zut', 'à', 'â', 'ça', 'ès', 
             'étaient', 'étais', 'était', 'étant', 'été', 'être', 'ô','a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 
             'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'qu']

stopWords = [unidecode(sw) for sw in stopWords]

# Création du stemmer
stemmer = SnowballStemmer('french')

In [ ]:
#-------------------------Creation of  functions for cleanning process-----------------------#

#find the hashtags 
def hashtags(tweet):
  hash = re.findall(r"#(\w+)", tweet)
  return hash

#remove username et @username
def remove_users(tweet):
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) 
    # remove tweeted at
    return tweet

#remove les links 
def remove_links(tweet):
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # remove bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

#remove non-ascii chars
def non_ascii(s):
    return "".join(i for i in s if ord(i)<128)

#lower case
def lower(doc):
    return doc.lower()

#remove les caractères spéciaux
def remove_special_characters(tweet):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\»\s]' 
    return re.sub(pat, '', tweet)

#remove stop words
def removeStopWords(str, stopWords):
    new_tweet = ' '.join([word for word in str.split() if word not in stopWords]) 
    return new_tweet

#remove email address
def email_address(doc):
    email = re.compile(r'[\w\.-]+@[\w\.-]+')
    return email.sub(r'',doc)

#remove blank space and special caracters
def punct(tweet):
    token=RegexpTokenizer(r'\w+')#regex
    tweet = token.tokenize(tweet)
    tweet= " ".join(tweet)
    return tweet 

#remove digits
def remove_digits(tweet):
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', tweet)



In [ ]:
#--------------------Test cleaning on a sample-------------------------#

sample = "je n'AIME pas ce ,^ #film$**LO-VE-LY@awatra http:www.awa.com awatra@gmail.com  : ;  ?!   123    ok"
punct(sample)

In [ ]:
#apply the functions to the dataframe 

df['hashtag'] = df.tweet.apply(func = hashtags)

#begin cleaning

df['new_tweet'] = df.tweet.apply(func = remove_users)
df['new_tweet'] = df.tweet.apply(func = remove_links)
df['new_tweet'] = df.tweet.apply(func = non_ascii)
df['new_tweet'] = df.tweet.apply(func = lower)
df['new_tweet'] = df.new_tweet.apply(func = email_address)
df['new_tweet'] = df.new_tweet.apply(func = removeStopWords , stopWords=stopWords)
df['new_tweet'] = df.new_tweet.apply(func = punct)
df['new_tweet'] = df.new_tweet.apply(func = remove_links)
df['new_tweet'] = df.new_tweet.apply(func = remove_digits)
df['new_tweet'] = df.new_tweet.apply(func = remove_special_characters)

In [ ]:
df.columns

In [ ]:
df['new_tweet'][12]

**IV. Normalisation**

In [ ]:
stemmer = SnowballStemmer('french') # Vous initialisez le stemmer en français

corpus1 = [" ".join([stemmer.stem(word) for word in doc.split()]) for doc in df['new_tweet']]

**V. VISUALISATION**

In [ ]:
df[['word_titles']].hist(figsize=(12,6), bins=8, xlabelsize=8, ylabelsize=9);
plt.title("Distribution of number of words in covid hashtags")

In [ ]:
plt.figure(figsize=(15,5))


corp = [len(doc.split()) for doc in corpus1]
corp

index=list(range(len(corp)))


plot = sns.barplot(x=index,y=corp, data=df)
    
plt.xlabel(' ')
plt.ylabel("nb")
plt.title("Répartition du nombre de mots par tweet") 

In [ ]:

# Tokenisation des articles
corpus = [doc for doc in corpus1]
list_tokens = ' '.join(corpus).split()

# Identification et décompte des bigrams
list_bigrams = list(ngrams(list_tokens, 2)) 
counter = Counter(list_bigrams)

# Création d'un dataframe contenant le top 20 bigramme les plus fréquents
df_plot = pd.DataFrame(counter.most_common(20))

# Création du graphique
plt.figure(figsize=(15,5))

plot = sns.barplot(x=df_plot[1], 
                    y=df_plot[0],
                    color='#44546a')

plt.xlabel('occurrence')
plt.ylabel("bigramme")
plt.title("Top 20 bigrammes catégorie") 

In [ ]:
all_words = ''.join([word for word in corpus1[0:10000]])
all_words 
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
plt.figure(figsize=(15,8))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title("les mots fréquemment utilisés dans les hashtags", weight='bold', fontsize=14)
plt.show()

In [ ]:
#-------------------most usefull hashtags------------------#
text = str(df.new_tweet)
wordcloud = WordCloud().generate(text)
stopwords = set(STOPWORDS)
wordcloud = WordCloud(max_font_size=80, max_words=60, background_color="white").generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show() 

**VI. TOPIC MODELLING**

In [ ]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stopWords)

In [ ]:
dtm = vectorizer.fit_transform(df['new_tweet'].values.astype('U')) # U convert the data set to unicode
dtm

In [ ]:
words = np.array(vectorizer.get_feature_names()) #visualising the number of feature the vectorizer gets
print(len(words)) 

In [ ]:
nmf_mod = NMF(n_components=10,random_state=42) #estimatte the topic model of the dataset 
nmf_mod.fit(dtm)

In [ ]:
for index, topic in enumerate(nmf_mod.components_):
  print(f"The TOP 10 words for Topic#{index}")
  print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])

In [ ]:
df_topics = nmf_mod.transform(dtm) #combine topics with the original data

In [ ]:
df_topics[1].argsort()

In [ ]:
df_topics[1].argmax()#index position of the most representative topic

In [ ]:
bbc_topiclabels = {0:'vaccin en israel',
                   1:'sentiments',
                   2:'impressions',
                   3:'augmentation contaminations dans le monde',
                   4:"fêtes de fin d'année période covid",
                   5:'nouveaux cas, Bordeaux, Quebec  ',
                   6:'colère médecin ouest France',
                   7:'tests PCR',
                   8:'période de grippe',
                   9:'masque obligatoire, omicron, covid'
                  }

In [ ]:
df['hash'] = df_topics.argmax(axis=1)

In [ ]:
df['topic_label'] = df['hash'].map(bbc_topiclabels)#each hashtag refers to which the topic?

In [ ]:
df2=df.groupby('hash').count()
df.head()

In [ ]:
df2

**VII CLUSTERING**

In [ ]:
dtm.toarray()

In [ ]:
#PCA
pca=PCA(n_components=2).fit(dtm.toarray())
X_pca=pca.transform(dtm.toarray())
plt.scatter(X_pca[:,0],X_pca[:,1])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA")

In [ ]:
#Determination du nombre optimal de cluster
wcss=[]

for i in range(1,11):
    model= KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0)
    model.fit(X_pca)
    wcss.append(model.inertia_)

#Graphique coude
plt.plot(range(1,11),wcss)
plt.title("Methode du coude")
plt.xlabel("Nombre de clusters")
plt.ylabel("Within Cluster Sum of Squares")
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=4).fit(X_pca)
y_pred=kmeans.predict(X_pca)

plt.scatter(X_pca[y_pred==0,0],y=X_pca[y_pred==0,1],c='red',alpha=0.6,label="cluster 1")
plt.scatter(X_pca[y_pred==1,0],y=X_pca[y_pred==1,1],c='blue',alpha=0.6,label="cluster 2")
plt.scatter(X_pca[y_pred==2,0],y=X_pca[y_pred==2,1],c='green',label="cluster 3")
plt.scatter(X_pca[y_pred==3,0],y=X_pca[y_pred==3,1],alpha=0.6,label="cluster 4")

plt.scatter(x=kmeans.cluster_centers_[:,0],y=kmeans.cluster_centers_[:,1],s=100,c="k",marker="X")
plt.legend()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-Means Clustering")

plt.show()